# Jimmy Wrangler, Data Explorer

Dalton Hahn (2762306)

## Donald Trump Approval Ratings (Job approval rating and trendline)
https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv

https://projects.fivethirtyeight.com/trump-approval-data/approval_topline.csv

## Exploratory Data Analysis
My idea is to examine the three datasets to see if there is any correlation or connection between President Trump's tweets and the lawsuits he and his company have been involved in.  Additionally, I would like to see if there is any correlation or connection between his tweets and his overall approval rating.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from statistics import mean, stdev

## Read in the Data

In [2]:
df_top = pd.read_csv("../data/external/approval_topline.csv")
df_poll = pd.read_csv("../data/external/approval_polllist.csv")

In [3]:
df_top.head()

,president,subgroup,modeldate,approve_estimate,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,timestamp
0,Donald Trump,Voters,9/10/2019,42.38458,46.79176,37.97740,53.32431,56.70696,49.94167,09:39:21 10 Sep 2019
1,Donald Trump,Adults,9/10/2019,39.89137,43.57331,36.20942,54.62294,58.93084,50.31503,09:38:22 10 Sep 2019
2,Donald Trump,All polls,9/10/2019,40.96740,45.68800,36.24680,54.15805,58.10293,50.21317,09:37:40 10 Sep 2019
3,Donald Trump,Voters,9/9/2019,42.62912,46.90967,38.34858,53.25327,56.73088,49.77565,18:04:24 9 Sep 2019
4,Donald Trump,Adults,9/9/2019,40.60160,44.39997,36.80324,54.26073,58.88997,49.63150,18:03:25 9 Sep 2019


In [4]:
df_poll.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,9/10/2019,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.245429,...,45.0,46.13177,43.29169,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,09:37:40 10 Sep 2019
1,Donald Trump,All polls,9/10/2019,1/20/2017,1/22/2017,Morning Consult,B-,1992.0,rv,0.944206,...,37.0,44.37482,38.65035,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,09:37:40 10 Sep 2019
2,Donald Trump,All polls,9/10/2019,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.226788,...,46.0,46.13177,44.29169,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,09:37:40 10 Sep 2019
3,Donald Trump,All polls,9/10/2019,1/20/2017,1/24/2017,Ipsos,B+,1632.0,a,0.224456,...,45.2,42.86502,43.93771,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,09:37:40 10 Sep 2019
4,Donald Trump,All polls,9/10/2019,1/22/2017,1/24/2017,Gallup,B,1500.0,a,0.212047,...,45.0,47.13177,43.29169,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49236,77248,1/25/2017,09:37:40 10 Sep 2019


## Plan to drop columns from approval_topline
- president
- approve_hi
- approve_lo
- disapprove_hi
- disapprove_lo
- timestamp

## Plan to drop columns from approval_pollist
- president
- pollster
- grade
- population
- weight
- approve
- disapprove
- multiversions
- tracking
- url
- poll_id
- question_id
- timestamp

### *Additionally, will drop any rows from approval_pollist with fewer than 10k samples*

In [5]:
# Dropping columns from approval_topline
df_top = df_top.drop(axis=1, columns=['president', 'approve_hi', 'approve_lo', 'disapprove_hi', 'disapprove_lo', 'timestamp'])
df_top.head()

,subgroup,modeldate,approve_estimate,disapprove_estimate
0,Voters,9/10/2019,42.38458,53.32431
1,Adults,9/10/2019,39.89137,54.62294
2,All polls,9/10/2019,40.96740,54.15805
3,Voters,9/9/2019,42.62912,53.25327
4,Adults,9/9/2019,40.60160,54.26073


In [6]:
# Dropping columns from approval_pollists
df_poll = df_poll.drop(axis=1, columns=['president', 'pollster', 'grade', 'population', 'weight', 'approve',
                                        'disapprove', 'multiversions', 'tracking', 'url', 'poll_id','question_id',
                                        'influence', 'timestamp'])
df_poll.head()

,subgroup,modeldate,startdate,enddate,samplesize,adjusted_approve,adjusted_disapprove,createddate
0,All polls,9/10/2019,1/20/2017,1/22/2017,1500.0,46.13177,43.29169,1/23/2017
1,All polls,9/10/2019,1/20/2017,1/22/2017,1992.0,44.37482,38.65035,1/23/2017
2,All polls,9/10/2019,1/21/2017,1/23/2017,1500.0,46.13177,44.29169,1/24/2017
3,All polls,9/10/2019,1/20/2017,1/24/2017,1632.0,42.86502,43.93771,3/1/2017
4,All polls,9/10/2019,1/22/2017,1/24/2017,1500.0,47.13177,43.29169,1/25/2017


### Seems like the createddate always appears a few days after the enddate of the poll, I'll keep this column for the datetime and drop the startdate and enddate.  I'll keep modeldate since that seems like it could match up with the approval_topline dataset

In [7]:
df_poll = df_poll.drop(axis=1, columns=['enddate', 'startdate'])
df_poll.head()

,subgroup,modeldate,samplesize,adjusted_approve,adjusted_disapprove,createddate
0,All polls,9/10/2019,1500.0,46.13177,43.29169,1/23/2017
1,All polls,9/10/2019,1992.0,44.37482,38.65035,1/23/2017
2,All polls,9/10/2019,1500.0,46.13177,44.29169,1/24/2017
3,All polls,9/10/2019,1632.0,42.86502,43.93771,3/1/2017
4,All polls,9/10/2019,1500.0,47.13177,43.29169,1/25/2017


In [8]:
# Dropping rows in df_poll that do not have >10k samplesize
df_poll_filtered = df_poll[df_poll['samplesize'] >= 10000]
df_poll_filtered.head()

,subgroup,modeldate,samplesize,adjusted_approve,adjusted_disapprove,createddate
95,All polls,9/10/2019,11512.0,40.57358,53.00161,2/20/2017
115,All polls,9/10/2019,10639.0,41.57358,53.00161,2/24/2017
140,All polls,9/10/2019,12273.0,42.57358,52.00161,11/28/2018
169,All polls,9/10/2019,12257.0,44.57358,51.00161,3/11/2017
198,All polls,9/10/2019,12133.0,43.57358,52.00161,3/17/2017


### Breaking dates in both datasets out by month, day and year

In [9]:
months = []
days = []
years = []

for row in df_top['modeldate']:
    month = row.split('/')[0]
    months.append(month)
    #print(month)
    day = row.split('/')[1]
    days.append(day)
    #print(day)
    year = row.split('/')[2].split(' ')[0]
    years.append(year)
    #print(year)

In [10]:
#Check that the splits I've created are the same size as the original data frame
print(len(days))
print(len(df_top['modeldate']))

2883
2883


In [11]:
df_top.insert(len(df_top.columns), 'M_Month', months, True)
df_top.insert(len(df_top.columns), 'M_Day', days, True)
df_top.insert(len(df_top.columns), 'M_Year', years, True)
df_top.head()

,subgroup,modeldate,approve_estimate,disapprove_estimate,M_Month,M_Day,M_Year
0,Voters,9/10/2019,42.38458,53.32431,9,10,2019
1,Adults,9/10/2019,39.89137,54.62294,9,10,2019
2,All polls,9/10/2019,40.96740,54.15805,9,10,2019
3,Voters,9/9/2019,42.62912,53.25327,9,9,2019
4,Adults,9/9/2019,40.60160,54.26073,9,9,2019


In [12]:
months = []
days = []
years = []

for row in df_poll_filtered['modeldate']:
    month = row.split('/')[0]
    months.append(month)
    #print(month)
    day = row.split('/')[1]
    days.append(day)
    #print(day)
    year = row.split('/')[2].split(' ')[0]
    years.append(year)
    #print(year)

In [13]:
c_months = []
c_days = []
c_years = []

for row in df_poll_filtered['createddate']:
    month = row.split('/')[0]
    c_months.append(month)
    #print(month)
    day = row.split('/')[1]
    c_days.append(day)
    #print(day)
    year = row.split('/')[2].split(' ')[0]
    c_years.append(year)
    #print(year)

In [14]:
#Check that the splits I've created are the same size as the original data frame
print(len(days))
print(len(df_poll_filtered['modeldate']))
print(len(c_days))
print(len(df_poll_filtered['createddate']))

9578
9578
9578
9578


In [15]:
df_poll_filtered.insert(len(df_poll_filtered.columns), 'M_Month', months, True)
df_poll_filtered.insert(len(df_poll_filtered.columns), 'M_Day', days, True)
df_poll_filtered.insert(len(df_poll.columns), 'M_Year', years, True)
df_poll_filtered.insert(len(df_poll.columns), 'C_Month', c_months, True)
df_poll_filtered.insert(len(df_poll.columns), 'C_Day', c_days, True)
df_poll.insert(len(df_poll.columns), 'C_Year', c_years, True)
df_poll.head()

,subgroup,modeldate,samplesize,adjusted_approve,adjusted_disapprove,createddate,M_Month,M_Day,M_Year,C_Month,C_Day,C_Year
0,All polls,9/10/2019,1500.0,46.13177,43.29169,1/23/2017,9,10,2019,1,23,2017
1,All polls,9/10/2019,1992.0,44.37482,38.65035,1/23/2017,9,10,2019,1,23,2017
2,All polls,9/10/2019,1500.0,46.13177,44.29169,1/24/2017,9,10,2019,1,24,2017
3,All polls,9/10/2019,1632.0,42.86502,43.93771,3/1/2017,9,10,2019,3,1,2017
4,All polls,9/10/2019,1500.0,47.13177,43.29169,1/25/2017,9,10,2019,1,25,2017


### Save out edited data into data/processed for use in analysis notebook

In [ ]:
df_top.to_csv("../data/processed/tweets_processed.csv")